In [2]:
from cnn-error-analysis import cifar10
from cnn-error-analysis import data_analysis as da
from cnn-error-analysis import visualization as vis
from cnn-error-analysis import evaluation as ev

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
# set paths to stored extracted features
fv_test_url = "D:/tmp/inceptionV1_finetuned_cifar10_version9/ckpt_204365_extracted/InceptionV1_CifarTest_fv.csv"
logits_test_url = "D:/tmp/inceptionV1_finetuned_cifar10_version9/ckpt_204365_extracted/InceptionV1_CifarTest_logits.csv"

fv_train_url = "D:/tmp/inceptionV1_finetuned_cifar10_version9/ckpt_204365_extracted/InceptionV1_CifarTrain_fv.csv"
logits_train_url = "D:/tmp/inceptionV1_finetuned_cifar10_version9/ckpt_204365_extracted/InceptionV1_CifarTrain_logits.csv"

In [4]:
current_analysis_folder = "F:/diplomarbeit_bjoern_moeller/analysis/NearestNeighbor_all_errors"

In [5]:
# load the data: Test Set
cifar_test_fv_df = cifar10.load_csv_into_df(fv_test_url)
cifar_test_logits_df = cifar10.load_csv_into_df(logits_test_url)
#Train Set
cifar_train_fv_df = cifar10.load_csv_into_df(fv_train_url)
cifar_train_logits_df = cifar10.load_csv_into_df(logits_train_url)

#concat
fv_df = pd.concat([cifar_test_fv_df, cifar_train_fv_df])
logits_df = pd.concat([cifar_test_logits_df, cifar_train_logits_df])
# predict and label column in logits df
logits_df = cifar10.add_predicted_classnr_column(logits_df)
logits_df = cifar10.add_true_classnr_column(logits_df)
logits_df.head(n=3)

#reset index
fv_df = fv_df.reset_index(drop=True)
logits_df = logits_df.reset_index(drop=True)

#slice
#x_fv_df = cifar10.slice_df_by_classes(fv_df,[current_class])
#x_logits_df = cifar10.slice_df_by_classes(logits_df,[current_class]) 

In [6]:
false_negatives = vis.get_false_negatives_for_class(logits_df, "truck")

df shape (6000, 13)


In [7]:
#distances, indices =  nearest_neighbors(fv_df, neighbors=20)

test = pd.read_csv(current_analysis_folder + "nn_indices.csv", sep=',', low_memory=False, error_bad_lines=False,header=None,skiprows=[0],index_col=0)

In [8]:
indices = test.to_numpy()
indices

array([[    0, 17142, 24578, ..., 23443, 11892, 41383],
       [    1, 47692,  6490, ..., 30936, 57655,  2692],
       [    2, 26620,  9343, ..., 36306, 49035,   310],
       ...,
       [59997, 35338, 28792, ..., 56072,  6871,  1064],
       [59998, 33923, 26211, ..., 47792, 44238, 15917],
       [59999, 27255, 24204, ..., 53984, 29764,  7052]], dtype=int64)

In [9]:
#Load Images
cifarDataManager = cifar10.CifarDataManager()
test_images = cifarDataManager.test.images
#x_test_images = test_images[x_fv_df.index.tolist(),:,:,:]

train_images = cifarDataManager.train.images
#x_train_images = train_images[x_logits_df.index.tolist(),:,:,:]


images = np.concatenate((test_images, train_images), axis=0)
images.shape

D:/tmp/cifar10_pythonversion/cifar-10-batches-py/data_batch_1
D:/tmp/cifar10_pythonversion/cifar-10-batches-py/data_batch_2
D:/tmp/cifar10_pythonversion/cifar-10-batches-py/data_batch_3
D:/tmp/cifar10_pythonversion/cifar-10-batches-py/data_batch_4
D:/tmp/cifar10_pythonversion/cifar-10-batches-py/data_batch_5
D:/tmp/cifar10_pythonversion/cifar-10-batches-py/test_batch


(60000, 32, 32, 3)

In [10]:
visualizer = vis.Visualizer(cifar10.class_names)

In [11]:
##### print output for one class #####
for i in [0,1,2,3,4,5,6,7,8]: #exclude class label for current class otherwise it will take some time (e.g. truck: exclude 9)
    if false_negatives[i] != []:
        img1 = vis.plot_nn_for_class(false_negatives[i],indices,logits_df, images, visualizer)
        img1.save(current_analysis_folder + 'image_truck_'+ str(i) + '.png')